In [7]:
from sys import prefix
import pandas as pd
import random
import os
import shutil
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import math
from z3 import *

class sudoku:
    
    def __init__(self, init=False, path='benchmarks', solver='sudoku_acs', execl='charts/base.xlsx', opt_path = '/INST_49x49/100'):
        self.path = path
        self.opt_path = path + opt_path
        self.solver = solver
        self.data_path = 'results/' + solver
        self.records = pd.read_excel(execl, header=[0], index_col = [0])
        if init:
            self.records = self.records.drop(index=self.records.index)
            self.get_info(path)
            
    def get_benchmarks(self, percentage=0):
        file_list = os.listdir(self.opt_path)
        target_path = self.opt_path[:-3] + str(percentage)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for file in tqdm(file_list, target_path):
            path = os.path.join(self.opt_path, file)
            with open(path) as f:
                grid = []
                order = f.readline()[:-1]
                is_sat = f.readline()
                square = int(order)*int(order)
                for i in range(square):
                    grid.append(f.readline().split())
                index = 0
                total = (100-percentage)*square*square/100
                while index < total:
                    x = random.randint(0, square-1)
                    y = random.randint(0, square-1)
                    if grid[x][y] != '-1':
                        grid[x][y] = '-1'
                        index += 1
                filename = file.split('_')
                filename[1] = str(percentage)
                filename = '_'.join(filename)
                with open(target_path + '/' + filename, 'w') as out:
                    out.write(order + '\n')
                    out.write(is_sat)
                    for i in range(square):
                        line = ''
                        for j in range(square):
                            line = line + grid[i][j] + '\t'
                        out.write(line[:-1] + '\n')

    def get_empty_benchmarks(self, path='benchmarks/INST_64x64/0'):
        square = int(path.split('/')[1].split('x')[-1])
        for i in range(100):
            filepath = f'{path}/inst{str(square)}x{str(square)}_0_{str(i)}.txt'
            # if not os.path.exists(filepath):
            #     os.makedirs(filepath)
            # filepath = path + '/inst64x64_0_' + str(i) + '.txt'
            with open(filepath, 'w') as f:
                f.write(str(int(math.sqrt(square))) + '\n') 
                f.write('1\n')
                for _ in range(square):
                    line = ''
                    for _ in range(square):
                        line = line + '-1' + '\t'
                    f.write(line[:-1] + '\n')

    def get_part_benchmarks(self, number=1, target_path='benchmarks_part'):
        order_list = os.listdir(self.path)
        if not os.path.exists(target_path):
            os.makedirs(target_path)
        for order in order_list:
            order_path = os.path.join(target_path, order)
            if not os.path.exists(order_path):
                os.makedirs(order_path)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                if not os.path.exists(percentage_path):
                    os.makedirs(percentage_path)
                v1 = order.split('_')[-1]
                v2 = str(i)
                if number != 0:
                    for index in range(number):
                        origin_path = f'benchmarks/inst{v1}_{v2}_{index}.txt'
                        shell = f'cp {origin_path} {percentage_path}'
                        # print(shell)
                        os.system(shell)

    def get_CP_benchmarks(self, path='benchmarks_CP_part'):
        order_list = os.listdir(path)
        for order in order_list:
            order_path = os.path.join(path, order)
            for i in range(0, 105, 5):
                percentage_path = os.path.join(order_path, str(i))
                file_list = os.listdir(percentage_path)
                for file in file_list:
                    file_path = os.path.join(percentage_path, file)
                    target_path = file_path.split('.')[0] + '.dzn'
                    shell = f'mv {file_path} {target_path}'
                    os.system(shell)
                    lines = []
                    with open(target_path) as f:
                        lines = f.readlines()
                    order = int(lines[0][:-1])
                    
                    lines[0] = f'S={str(order)};\n'
                    lines[1] = 'start=[|\n'
                    for i in range(order*order-1):
                        lines[i+2] = lines[i+2].strip('\t\n').replace('\t', ',\t') + '|\n'
                    lines[-1] = lines[-1].strip('\t\n').replace('\t', ',\t') + '|];\n'
                    with open(target_path, 'w') as f:
                        f.writelines(lines)
    
    def get_SMT_benchmarks(self, path='benchmarks_part'):
        shell = f'rm -rf benchmarks_SAT'
        os.system(shell)
        shell = f'cp -rf {path} benchmarks_SAT'
        os.system(shell)
        order_list = os.listdir('benchmarks_SAT')
        for order in order_list:
            order_path = os.path.join('benchmarks_SAT', order)
            origin_path = ''
            for percentage in tqdm(range(0, 105, 5), desc=order_path):
                percentage_path = os.path.join(order_path, str(percentage))
                file_list = os.listdir(percentage_path)
                for file in file_list:
                    if percentage == 0:
                        file_path = os.path.join(percentage_path, file)
                        target_path = file_path.split('.')[0] + '.smt2'
                        origin_path = target_path
                        shell = f'mv {file_path} {target_path}'
                        os.system(shell)
                        lines = []
                        with open(target_path) as f:
                            lines = f.readlines()
                        order = int(lines[0][:-1])
                        n = order*order
                        
                        A = []
                        for i in range(n):
                            A.append(lines[i+2].strip('\t\n').split('\t'))
                        X = [ [ [ Bool("x_%s_%s_%s" % (i+1, j+1, v+1)) for v in range(n) ] for j in range(n) ] for i in range(n) ]
                        domain_c = []
                        for i in range(n):
                            for j in range(n):
                                domain_c += [ AtMost(*[X[i][j][v] for v in range(n)], 1) ]
                                domain_c += [ AtLeast(*[X[i][j][v] for v in range(n)], 1) ] 
                        dists_c = []
                        for i in range(n):
                            for v in range(n):
                                dists_c += [ AtMost(*[X[i][j][v] for j in range(n)], 1) ]
                                dists_c += [ AtLeast(*[X[i][j][v] for j in range(n)], 1) ]
                        for j in range(n):
                            for v in range(n):
                                dists_c += [ AtMost(*[X[i][j][v] for i in range(n)], 1) ]
                                dists_c += [ AtLeast(*[X[i][j][v] for i in range(n)], 1) ]
                        for x in range(order):
                            for y in range(order):
                                for v in range(n):
                                    dists_c += [ AtMost(*[X[x*order + x1][y*order + y1][v] for x1 in range(order) for y1 in range(order)], 1) ]
                                    dists_c += [ AtLeast(*[X[x*order + x1][y*order + y1][v] for x1 in range(order) for y1 in range(order)], 1) ]
                        
                        sudoku_c = domain_c + dists_c
                        s = Solver()
                        s.add(sudoku_c)
                        smt = s.to_smt2().split('\n')
                        smt = [i+'\n' for i in smt]
                        with open(target_path, 'w') as f:
                            f.writelines(smt)
                
                        use_path = file_path.split('.')[0] + '_use.smt2'
                        shell = f'cp {target_path} {use_path}'
                        os.system(shell)
                        
                    else:
                        file_path = os.path.join(percentage_path, file)
                        target_path = file_path.split('.')[0] + '.smt2'
                        shell = f'mv {file_path} {target_path}'
                        os.system(shell)
                        shell = f'cp {origin_path} {target_path}'
                        os.system(shell)
                        use_path = file_path.split('.')[0] + '_use.smt2'
                        shell = f'cp {target_path} {use_path}'
                        os.system(shell)

    def get_SAT_benchmarks(self, path='benchmarks'):
        pass    

    def get_info(self, path):
        folder_list = os.listdir(path)
        for folder in folder_list:
            new_path = os.path.join(path, folder)
            if os.path.isdir(new_path):
                self.get_info(new_path)
            else:
                filename = new_path.split('/')[-1]
                category = filename.split('_')[0]
                percentage = filename.split('_')[1]
                self.records.loc[len(self.records.index)] = [filename, category, percentage]
        
    def save_to(self, filename):
        self.records.to_excel(filename)
        
    def arrange(self):
        # for filename in tqdm(folder_list, desc=folder_path):
        category_list = os.listdir(self.data_path)
        for folder in category_list:
            folder_path = os.path.join(self.data_path, folder)
            file_list = os.listdir(folder_path)
            for file in tqdm(file_list, desc=folder_path):
                path = os.path.join(folder_path, file)
                with open(path) as f:
                    try:
                        if self.solver == 'sudoku_acs':
                            while f.readline():
                                result = f.readline()[:-1]
                                if result == '0':
                                    result = 'sat'
                                elif result == '1':
                                    result = 'timeout'
                                time = f.readline()[:-1]
                                if result == 'timeout':
                                    time = '1000.000000'
                                info = f.readline()
                                # time = info.split(' : ')[-1][:-5]
                                name = info.split(' : ')[0].split('/')[-1]
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_ort':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                if info == 'SAT':
                                    result = 'sat'
                                    info = f.readline()
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                name = info.split(' ')[0].split('/')[-1]
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_ils':
                            while f.readline():
                                result = f.readline()[:-1]
                                info = f.readline()[:-1]
                                if result == '0':
                                    result = 'sat'
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                else:
                                    result = 'timeout'
                                    time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_lsc':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_csp':
                            while f.readline():
                                result = f.readline()[:-1]
                                time = '1000.000000'
                                if result[:3] == 'SAT':
                                    result = 'sat'
                                    f.readline()
                                    f.readline()
                                elif result[:3] == '%% ':
                                    result = 'timeout'
                                    f.readline()
                                info = f.readline()
                                if result == 'sat':
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                name = info.split(' : ')[0].split('/')[-1].split('.')[0] + '.txt'
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_sat':
                            while f.readline():
                                info = ''
                                result = f.readline()[:-1]
                                time = '1000.000000'
                                if result[:3] == 's S':
                                    result = 'sat'
                                    info = f.readline()
                                else:
                                    info = result
                                    result = 'timeout'
                                if result == 'sat':
                                    time = int(info.split(' : ')[-1].split(' ')[0]) / 1000.0
                                name = info.split(' : ')[0].split('/')[-1].split('.')[0] + '.txt'
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_1':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_2':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_3':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_4':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_5':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_step':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_full':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                    f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_swap':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_test_tabu':
                            while f.readline():
                                info = f.readline()[:-1]
                                result = 'timeout'
                                time = '1000.000000'
                                name = info.split(' ')[0].split('/')[-1]
                                if 'find answer' in info:
                                    result = 'sat'
                                    time = info.split(': ')[-1]
                                f.readline()
                                # print(name)
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_rule1234':
                            while f.readline():
                                info = f.readline()[:-1]
                                name = info.split(' ')[0].split('/')[-1]
                                result = 'sat'
                                time = info.split(': ')[-1]
                                f.readline()
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_rule1':
                            while f.readline():
                                info = f.readline()[:-1]
                                name = info.split(' ')[0].split('/')[-1]
                                result = 'sat'
                                time = info.split(': ')[-1]
                                f.readline()
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                        elif self.solver == 'sudoku_rule234':
                            while f.readline():
                                info = f.readline()[:-1]
                                name = info.split(' ')[0].split('/')[-1]
                                result = 'sat'
                                time = info.split(': ')[-1]
                                f.readline()
                                self.records.loc[self.records['filename'] == name, ['result_' + self.solver, 'time_' + self.solver]] = [result, float(time)]
                    except:
                        print(path)
    
    def arrange_hard(self):
        sat_num = [[0 for i in range(100)] for i in range(4)]
        total_time = [[0 for i in range(100)] for i in range(4)]
        clash_num = [[0 for i in range(100)] for i in range(4)]
        for result in tqdm(os.listdir('results')):
            result_path = os.path.join('results', result)
            category_list = os.listdir(result_path)
            for category in category_list:
                category_path = os.path.join(result_path, category)
                file_list = os.listdir(category_path)
                for file in file_list:
                    path = os.path.join(category_path, file)
                    with open(path) as f:
                        # try:
                        while f.readline():
                            info = f.readline()[:-1]
                            # print(result, '-', info)
                            if 'find answer' in info:
                                if '36x36_50' in info:
                                    time = float(info.split(': ')[-1])
                                    info = f.readline()
                                    index = int(info.split('36x36_50_')[-1].split('.')[0])
                                    sat_num[0][index] += 1
                                    total_time[0][index] += time
                                elif '49x49_50' in info:
                                    time = float(info.split(': ')[-1])
                                    info = f.readline()
                                    index = int(info.split('49x49_50_')[-1].split('.')[0])
                                    sat_num[1][index] += 1
                                    total_time[1][index] += time
                                elif '64x64_50' in info:
                                    time = float(info.split(': ')[-1])
                                    info = f.readline()
                                    index = int(info.split('64x64_50_')[-1].split('.')[0])
                                    sat_num[2][index] += 1
                                    total_time[2][index] += time
                                elif '64x64_60' in info:
                                    time = float(info.split(': ')[-1])
                                    info = f.readline()
                                    index = int(info.split('64x64_60_')[-1].split('.')[0])
                                    sat_num[3][index] += 1
                                    total_time[3][index] += time   
                            else:
                                clash = int(info.split(': ')[-1])
                                info = f.readline()
                                if '36x36_50' in info:
                                    index = int(info.split('36x36_50_')[-1].split('.')[0])
                                    clash_num[0][index] += clash
                                elif '49x49_50' in info:
                                    index = int(info.split('49x49_50_')[-1].split('.')[0])
                                    clash_num[1][index] += clash
                                elif '64x64_50' in info:
                                    index = int(info.split('64x64_50_')[-1].split('.')[0])
                                    clash_num[2][index] += clash
                                elif '64x64_60' in info:
                                    index = int(info.split('64x64_60_')[-1].split('.')[0])
                                    clash_num[3][index] += clash
                        # except:
                        #     print(path)
        for i in range(4):
            for j in range(100):
                if sat_num[i][j] == 0:
                    print(i, '-', j, ', sat_num: ', sat_num[i][j])
                    print(i, '-total_time: ', total_time[i][j])
                    print(i, '-clash_num: ', clash_num[i][j])
        

In [ ]:
# a = sudoku()
# a.arrange_hard()

初始时，生成未填充的sudoku，用于求解得到完全填充的sudoku

In [9]:
a = sudoku(opt_path='/INST_100x100/100')
a.get_empty_benchmarks(path='benchmarks/INST_121x121/0')

初始时，由完全填充的sudoku，删除部分值得到部分填充的benchmarks

In [ ]:
# a = sudoku(opt_path='/INST_81x81/100')
# for i in range(0, 100, 5):
#     a.get_benchmarks(percentage=i)

获取部分benchmarks（每个类型只有部分文件）用于测试

In [ ]:
a = sudoku()
a.get_part_benchmarks(number=0,target_path='benchmarks_SAT')
# for i in range(0, 105, 5):
#     print(i)

获取用于CP的benchmarks

In [ ]:
a = sudoku()
a.get_CP_benchmarks(path='benchmarks_part_CP')

获取用于SAT的benchmarks

In [ ]:
a = sudoku()
a.get_SAT_benchmarks(path='benchmarks_part')

初始化表格

In [ ]:
a = sudoku(init=True)
a.save_to('charts/base.xlsx')

sudoku_acs 生成表格

In [ ]:
c = sudoku(solver='sudoku_acs')
c.arrange()
c.save_to('charts/sudoku_acs.xlsx')

sudoku_csp 生成表格

In [ ]:
c = sudoku(solver='sudoku_csp', execl='charts/sudoku_acs.xlsx')
c.arrange()
c.save_to('charts/sudoku_csp.xlsx')

sudoku_sat 生成表格

In [ ]:
c = sudoku(solver='sudoku_sat', execl='charts/sudoku_csp.xlsx')
c.arrange()
c.save_to('charts/sudoku_sat.xlsx')

sudoku_ils 生成表格

In [ ]:
c = sudoku(solver='sudoku_ils', execl='charts/sudoku_acs.xlsx')
c.arrange()
c.save_to('charts/sudoku_ils.xlsx')

sudoku_ort 生成表格

In [ ]:
c = sudoku(solver='sudoku_ort', execl='charts/sudoku_ils.xlsx')
c.arrange()
c.save_to('charts/sudoku_ort.xlsx')

sudoku_test 生成表格

In [ ]:
c = sudoku(solver='sudoku_test', execl='charts/sudoku_ort.xlsx')
c.arrange()
c.save_to('charts/sudoku_test.xlsx')

sudoku_lsc 生成表格

In [ ]:
c = sudoku(solver='sudoku_lsc', execl='charts/sudoku_test.xlsx')
c.arrange()
c.save_to('charts/sudoku_lsc.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_1', execl='charts/sudoku_test.xlsx')
c.arrange()
c.save_to('charts/sudoku_1.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_2', execl='charts/sudoku_1.xlsx')
c.arrange()
c.save_to('charts/sudoku_2.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_3', execl='charts/sudoku_2.xlsx')
c.arrange()
c.save_to('charts/sudoku_3.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_4', execl='charts/sudoku_3.xlsx')
c.arrange()
c.save_to('charts/sudoku_4.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_5', execl='charts/sudoku_4.xlsx')
c.arrange()
c.save_to('charts/sudoku_5.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test')
c.arrange()
c.save_to('charts/sudoku_test.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_step', execl='charts/sudoku_test.xlsx')
c.arrange()
c.save_to('charts/sudoku_step.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_tabu', execl='charts/sudoku_step.xlsx')
c.arrange()
c.save_to('charts/sudoku_tabu.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_swap', execl='charts/sudoku_tabu.xlsx')
c.arrange()
c.save_to('charts/sudoku_swap.xlsx')

In [ ]:
c = sudoku(solver='sudoku_test_full', execl='charts/sudoku_swap.xlsx')
c.arrange()
c.save_to('charts/sudoku_full.xlsx')

In [10]:
c = sudoku(solver='sudoku_rule1234')
c.arrange()
c.save_to('charts/sudoku_rule1234.xlsx')

results/sudoku_rule1234/INST_49x49:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule1234/INST_36x36:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule1234/INST_81x81:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule1234/INST_64x64:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule1234/INST_16x16:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1234/INST_25x25:   0%|          | 0/11 [00:00<?, ?it/s]

In [11]:
c = sudoku(solver='sudoku_rule1', execl='charts/sudoku_rule1234.xlsx')
c.arrange()
c.save_to('charts/sudoku_rule1.xlsx')

results/sudoku_rule1/INST_49x49:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1/INST_36x36:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1/INST_81x81:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1/INST_64x64:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1/INST_16x16:   0%|          | 0/11 [00:00<?, ?it/s]

results/sudoku_rule1/INST_25x25:   0%|          | 0/11 [00:00<?, ?it/s]

In [12]:
c = sudoku(solver='sudoku_rule234', execl='charts/sudoku_rule1.xlsx')
c.arrange()
c.save_to('charts/sudoku_rule234.xlsx')

results/sudoku_rule234/INST_49x49:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule234/INST_36x36:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule234/INST_81x81:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule234/INST_64x64:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule234/INST_16x16:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_rule234/INST_25x25:   0%|          | 0/10 [00:00<?, ?it/s]

In [13]:
c = sudoku(solver='sudoku_ort', execl='charts/sudoku_rule234.xlsx')
c.arrange()
c.save_to('charts/sudoku_rule.xlsx')

results/sudoku_ort/INST_49x49:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_9x9:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_36x36:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_81x81:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_64x64:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_16x16:   0%|          | 0/10 [00:00<?, ?it/s]

results/sudoku_ort/INST_25x25:   0%|          | 0/10 [00:00<?, ?it/s]

In [14]:
c = sudoku(solver='sudoku_test', execl='charts/sudoku_rule.xlsx')
c.arrange()
c.save_to('charts/sudoku_rule.xlsx')

results/sudoku_test/INST_49x49:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_9x9:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_36x36:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_81x81:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_64x64:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_16x16:   0%|          | 0/21 [00:00<?, ?it/s]

results/sudoku_test/INST_25x25:   0%|          | 0/21 [00:00<?, ?it/s]